In [1]:
import os, time
import requests
import json
from datetime import datetime

In [2]:
# Config section:
# Define the API response JSON storage path:
api_resp_path = '/FileStore/tables/Citi-Bike/api_response'
# If dir doesnt exist then mkdir, else ignore:
dbutils.fs.mkdirs(api_resp_path)

Out[2]: True

In [3]:
# Define Realtime CitiBike API URLs:
system_information = "https://gbfs.citibikenyc.com/gbfs/en/system_information.json"
station_information = "https://gbfs.citibikenyc.com/gbfs/en/station_information.json"
station_status = "https://gbfs.citibikenyc.com/gbfs/en/station_status.json"
free_bike_status = "https://gbfs.citibikenyc.com/gbfs/en/free_bike_status.json"
system_hours = "https://gbfs.citibikenyc.com/gbfs/en/system_hours.json"
system_calendar = "https://gbfs.citibikenyc.com/gbfs/en/system_calendar.json"
system_regions = "https://gbfs.citibikenyc.com/gbfs/en/system_regions.json"
system_alerts = "https://gbfs.citibikenyc.com/gbfs/en/system_alerts.json"

##### Create API Response Path - Dir & File(s)
##### Call RealTime CitiBike APIs and GET JSONs - each minute:

In [5]:
# Keep fetching API response data every 90 seconds infinitely:
sleep_time = 60

while True:
  now = datetime.now() # current date and time
  fmt_now = now.strftime("%Y%m%d_%H-%M-%S")
  #print("date and time:",fmt_now)	
  # Create the JSON file:
  try:
    print('---------------------------------------------------')
    print('Creating empty JSON response file at start of Loop.')
    dbutils.fs.put(f"{api_resp_path}/station_status_{fmt_now}.json", "")
  except:
    print('File already exists')
  
  # Call API & get JSON response:
  #----------------------------------------------------------
  resp = requests.get(station_status)
  if resp.status_code != 200:
      # This means something went wrong
      raise ApiError(f'GET /tasks/ {resp.status_code}')

  print("Response Status Code : ", resp.status_code)
  resp_json_str = resp.content.decode("utf-8")
  print("Byte size of JSON Response: ", len(resp_json_str))
  #----------------------------------------------------------
  
  with open(f"/dbfs/{api_resp_path}/station_status_{fmt_now}.json","w") as f:
    f.write(resp_json_str)
  
  print(f"going to sleep for {sleep_time} seconds")
  time.sleep(sleep_time)

In [6]:
# Verification of pipeline step
#------------------------------
#Verify write by Reading JSON file with Python OS file read library:
#For standard File library of Python: prepend with /dbfs
#https://docs.databricks.com/data/databricks-file-system.html
'''
with open(f"/dbfs/{api_resp_path}/station_status.json", 'r') as f:
    data = f.read()
print(type(data))
print(data)
'''